In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


In [2]:
os.chdir("/Users/richcriticism/Documents/Documents/Bhavya_code/machine learning/CorrectPose2")

In [3]:
%pwd

'/Users/richcriticism/Documents/Documents/Bhavya_code/machine learning/CorrectPose2'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareModelConfig:
    root_dir: Path
    model_path: Path
    updated_model_path : Path
    params_learning_rate: float
    params_weights: str
    params_classes: int
    params_batch_size: int


/Users/richcriticism/Documents/Documents/Bhavya_code/machine learning/CorrectPose2/src/cnnClassifier

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [7]:
class ConfigurationManager:
        def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(CONFIG_FILE_PATH)
            self.params = read_yaml(PARAMS_FILE_PATH)
            create_directories([self.config.artifacts_root])
        
        def get_prepare_model_config(self) -> PrepareModelConfig:
            config = self.config.prepare_model

            create_directories([config.root_dir])
      

            prepare_model_config = PrepareModelConfig(
                  root_dir = Path(config.root_dir),
                  model_path = Path(config.model_path),
                  updated_model_path = Path(config.updated_model_path),
                  params_learning_rate = self.params.LEARNING_RATE,
                  params_weights = self.params.WEIGHTS,
                  params_classes = self.params.CLASSES,
                  params_batch_size = self.params.BATCH_SIZE
                  )
            
            return prepare_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import keras
from tensorflow.keras import Sequential
from keras.layers import Dense, Dropout, Activation, ReLU, Input, Flatten
from tensorflow.keras.models import Model

In [9]:
class PrepareModel:
    def __init__(self, config: PrepareModelConfig):
        self.config = config
        self.model = None

    
    def _prepare_model(self, learning_rate, classes, batch_size):

        input_shape = (batch_size, 6)
        
        base_model = Sequential([
            Input(shape = input_shape),
            Dense(6, activation='relu'),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(1, activation = "sigmoid")
            ])

        base_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])

        
        base_model.summary()

        dummy_input = tf.zeros((1,) + input_shape)

        # Call the base model on the dummy input data
        _ = base_model(dummy_input)

        # prediction = tf.keras.layers.Dense(
        #     units = classes,
        #     activation ='sigmoid'
        # )(base_model.output)

        # modelinputs = Input(shape = input_shape)

        # final_model = Model(
        #     inputs = modelinputs,
        #     outputs = prediction
        # )(base_model.output)
         
        base_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])
         
        base_model.summary()
         
        return base_model
    
    def update_model(self):

        tf.keras.backend.clear_session()
        
        self.final_model = self._prepare_model(
            classes = self.config.params_classes,
            learning_rate = self.config.params_learning_rate,
            batch_size=self.config.params_batch_size
        )

        self.save_model(path = self.config.model_path, final_model=self.final_model)

    
    @staticmethod
    def save_model(path, final_model: tf.keras.Model):
        final_model.save(path)

In [10]:
try:
    config = ConfigurationManager()
    prepare_model_config = config.get_prepare_model_config()
    prepare_model = PrepareModel(config=prepare_model_config)
    
    # Call update_model without passing any additional arguments
    prepare_model.update_model()

except Exception as e:
    print("An error occurred:", str(e))
    raise e

[2024-04-03 20:25:05,812: INFO : common : yaml file : config/config.yaml loaded successfully]
[2024-04-03 20:25:05,814: INFO : common : yaml file : params.yaml loaded successfully]
[2024-04-03 20:25:05,815: INFO : common : creating directory at path: artifacts]
[2024-04-03 20:25:05,816: INFO : common : creating directory at path: artifacts/prepare_base_model]


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 12, 6)          │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12, 32)         │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 12, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12, 1)          │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 811 (3.17 KB)

 Trainable params: 811 (3.17 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 12, 6)          │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12, 32)         │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 12, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12, 1)          │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 811 (3.17 KB)

 Trainable params: 811 (3.17 KB)

 Non-trainable params: 0 (0.00 B)

[2024-04-03 20:25:05,928: WARNING : saving_api : You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
